### call-your-own-api-inside-ai-assistant-openai-function-calling

In [1]:
import dotenv
%load_ext dotenv
%dotenv

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from openai import OpenAI
client = OpenAI()

In [ ]:
# [81.206732,26.706858,81.211581,26.710769]

In [38]:
get_crop_yeild = {
    "name" : "get_crop_yeild",
    "description" : "Function takes in Date and Location and give crop yeild prediction",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "Date": {
                "type":"string",
                "description" : "Datetime for which data needs to be fetched"
            },
            "Location":{
                "type" : "string",
                "description" : "Location for which data needs to be fetched"
            },
            "BBOX":{
                "type" : "array",
                "items": {
                            "type": "number"
                            },
                "description" : "Bounding Box for the Location for which data needs to be fetched"
            }
        },
        "required" : ["Date","BBOX"]
    }
}

In [39]:
assistant = client.beta.assistants.create(
    instructions="You are a bot who works like a code-interpreter and task executor",
    model="gpt-3.5-turbo-1106",
    tools = [{"type":"function", "function":get_crop_yeild}]
)
assistant

Assistant(id='asst_BLkQjiOrw4Y5htKtBwuDhgKm', created_at=1718176009, description=None, instructions='You are a bot who works like a code-interpreter and task executor', metadata={}, model='gpt-3.5-turbo-1106', name=None, object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_crop_yeild', description='Function takes in Date and Location and give crop yeild prediction', parameters={'type': 'object', 'properties': {'Date': {'type': 'string', 'description': 'Datetime for which data needs to be fetched'}, 'Location': {'type': 'string', 'description': 'Location for which data needs to be fetched'}, 'BBOX': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Bounding Box for the Location for which data needs to be fetched'}}, 'required': ['Date', 'BBOX']}), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

In [40]:
thread = client.beta.threads.create()
thread

Thread(id='thread_KOjf2TAkHAZQV90GyTxxCMrg', created_at=1718176011, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [42]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='What is the crop yield for area with [80.118914,27.826019,80.180197,27.885513] on 2024-06-09 ?'
)

In [43]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)


In [44]:
run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id,
)

In [45]:
assistant

Assistant(id='asst_BLkQjiOrw4Y5htKtBwuDhgKm', created_at=1718176009, description=None, instructions='You are a bot who works like a code-interpreter and task executor', metadata={}, model='gpt-3.5-turbo-1106', name=None, object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='get_crop_yeild', description='Function takes in Date and Location and give crop yeild prediction', parameters={'type': 'object', 'properties': {'Date': {'type': 'string', 'description': 'Datetime for which data needs to be fetched'}, 'Location': {'type': 'string', 'description': 'Location for which data needs to be fetched'}, 'BBOX': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Bounding Box for the Location for which data needs to be fetched'}}, 'required': ['Date', 'BBOX']}), type='function')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=None), top_p=1.0)

In [46]:
run.status

'requires_action'

In [47]:
import time
while run.status not in ["completed", "failed","requires_action"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [51]:
tool_calls = run.required_action.submit_tool_outputs.tool_calls

#print(tool_calls)

In [54]:

if tool_calls[0].function.name == "get_crop_yeild":

    import json
    import datetime
    import numpy as np
    from sentinelhub import SHConfig, CRS, BBox, DataCollection, MimeType, WcsRequest, CustomUrlParam
    import rasterio
    import requests
    from rasterstats import zonal_stats
    import geopandas as gpd
    import pyproj


    # Extract arguments from the input
    args = json.loads(tool_calls[0].function.arguments)
    Date, BBOX = args['Date'], arg['BBOX']
    
    def cropYeild(date: str, bbox: list) -> str:
        """Calculate crop yield for date and bbox provided"""

        INSTANCE_ID = '924b0ee0-f5a0-4b73-b2fb-f4edb8ab196d'  # Replace with your actual instance ID

        if INSTANCE_ID:
            config = SHConfig()
            config.instance_id = INSTANCE_ID
        else:
            config = None

        formatted_time = str(date)

        def coordinates_calc():
            # Define the bounding box
            # min_max = [x_min, y_min, x_max, y_max]
            min_max = BBOX
            print(min_max)
            src_crs = pyproj.CRS('EPSG:4326')  # WGS 84
            dst_crs = pyproj.CRS('EPSG:32644')  # UTM 44N

            # Define the coordinate transformation
            transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)

            # Transform the coordinates to the UTM projection and units
            x_min, y_min = transformer.transform(min_max[0], min_max[1])
            x_max, y_max = transformer.transform(min_max[2], min_max[3])

            # Define the bounding box in the UTM projection and units
            bbbox = [x_min, y_min, x_max, y_max]
            # print(coordinates)
            # print((bbbox))
            inversed_geojson_data = geojson_data.copy()

        # Reverse the coordinate pairs using list comprehension
            inversed_coordinates = [
                [coord[1], coord[0]] for coord in geojson_data["geometry"]["coordinates"][0]
            ]
            # Replace the coordinates in the copied inversed_geojson_data
            inversed_geojson_data["geometry"]["coordinates"] = [inversed_coordinates]
            return bbbox


        def dimensions():
            extent = coordinates_calc()
            bbox = BBox(bbox=extent, crs=CRS.WGS84)
            FAPAR = fapar_layer_call(bbox)
            transform = rasterio.transform.from_bounds(extent[0], extent[1], extent[2], extent[3], FAPAR.shape[1], FAPAR.shape[0])
            return bbox, transform

        def fapar_layer_call(bbox):
            wcs_request = WcsRequest(
                data_collection=DataCollection.SENTINEL2_L2A,
                layer='FAPAR',
                bbox=bbox,
                time=formatted_time,
                resx='10m',
                resy='10m',
                image_format=MimeType.TIFF,
                custom_url_params={CustomUrlParam.SHOWLOGO: False},
                config=config
            )
            return wcs_request.get_data()[0]

        def LSWI_layer_call(bbox):
            wcs_request = WcsRequest(
                data_collection=DataCollection.SENTINEL2_L2A,
                layer='LSWI',
                bbox=bbox,
                time=formatted_time,
                resx='10m',
                resy='10m',
                image_format=MimeType.TIFF,
                custom_url_params={CustomUrlParam.SHOWLOGO: False},
                config=config
            )
            return wcs_request.get_data()[0]

        def w_calc():
            bbox, transform = dimensions()
            LSWI = LSWI_layer_call(bbox)
            LSWI_max = np.amax(LSWI)
            w = (1 - LSWI) / (1 + LSWI_max)
            return w, transform

        def lue_calc():
            e0 = 3.22
            w, transform = w_calc()
            LUE = e0 * w
            return LUE, transform

        def ghi_calc():
            bbox, transform = dimensions()
            centroid_x = (bbox.min_x + bbox.max_x) / 2
            centroid_y = (bbox.min_y + bbox.max_y) / 2
            API_KEY = 'your-openweathermap-api-key'  # Replace with your actual OpenWeatherMap API key
            response = requests.get(f"https://api.openweathermap.org/energy/1.0/solar/data?lat={centroid_y}&lon={centroid_x}&date={formatted_time}&appid={API_KEY}")
            data = response.json()
            GHI = 0
            for i in data["irradiance"]["daily"]:
                GHI = i['clear_sky']['ghi']
            GHI = GHI / 1000 * 3.6
            return GHI

        def npp_calc():
            bbox, transform = dimensions()
            FAPAR = fapar_layer_call(bbox)
            GHI = ghi_calc()
            LUE, transform = lue_calc()
            NPP = FAPAR * GHI * 0.5 * LUE
            NPP[NPP < 0] = 0
            return NPP

        def cy_calc():
            NPP = npp_calc()
            cy = NPP * 0.8 * 10
            return cy

        def save_tiff(data, transform, path):
            with rasterio.open(path, 'w', driver='GTiff', width=data.shape[1], height=data.shape[0], count=1, dtype=data.dtype, crs='EPSG:4326', transform=transform) as dst:
                dst.write(data, 1)

        cy = cy_calc()
        save_tiff(cy, dimensions()[1], 'data/cy.tiff')

        shp = gpd.read_file("data/output.shp").to_crs(epsg=4326)
        with rasterio.open("data/cy.tiff") as src:
            out_image, out_transform = rasterio.mask.mask(src, shp.geometry, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({"driver": "GTiff", "height": out_image.shape[1], "width": out_image.shape[2], "transform": out_transform})

        with rasterio.open("data/cy.tiff", "w", **out_meta) as dest:
            dest.write(out_image)

        def zonal_stats_calc():
            stats = zonal_stats("data/output.shp", "data/cy.tiff", stats="mean")
            return stats[0]['mean']

        cy_mean = zonal_stats_calc()
        return round(cy_mean, 2)

    response = cropYeild(Date, BBOX)

    tool_outputs = []
    tool_outputs.append({"tool_call_id": tool_calls[0]['id'], "output": response})
    print(tool_outputs)


c:\Users\ARMS4AI\.conda\envs\jarvis4ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['BBOX']


C:\Users\ARMS4AI\AppData\Roaming\Python\Python311\site-packages\sentinelhub\config.py:53: SHDeprecationWarning: The parameter `sh_auth_base_url` of `SHConfig` has been replaced with `sh_token_url`. Please update your configuration, for now the parameters were automatically adjusted to `sh_token_url = sh_auth_base_url + '/oauth/token'`.
  warnings.warn(


IndexError: list index out of range

In [64]:
import json
from sentinelhub import SHConfig
import datetime
import pandas as pd
import numpy as np
from sentinelhub import CRS, BBox, DataCollection, MimeType, WcsRequest
from sentinelhub import CustomUrlParam
import geopandas as gpd
import rasterio
import rasterio.mask
from sentinelhub import CRS, BBox, DataCollection, MimeType, WcsRequest, CustomUrlParam
import requests
import json
import geopandas as gpd
from rasterstats import zonal_stats
from datetime import datetime
import warnings

from sentinelhub import SHConfig
warnings.filterwarnings("ignore")

# Extract arguments from the input
args = json.loads(tool_calls[0].function.arguments)
Date, BBOX = args['Date'], args['BBOX']

if tool_calls[0].function.name == "get_crop_yeild":
    # place to make a call and fetch data
    print("crop-yield-script")

    ### INPUT DATA 
    def cropYeild(date: str, bbox: list) -> str:
        def input_data():
            input_date = date

            # crop = data['selectedCrop']
            crop =  "Sugarcane"

            print(f"Input Data fetched \ndate: {input_date}\ncrop: {crop}")
            
            return input_date, crop


        ### SENTINEL HUB INSTANCE INFORMATION

        
        config = SHConfig()
        config.instance_id = 'c7deb951-735b-4978-8993-fbe59b03ae15'
        config.sh_client_id = 'dfff811c-7c59-4908-87c1-fb426cd4a768'
        config.sh_client_secret = 'VgL4yiU62gC6ZW5AqIYMvfh16y20edPl'
        config.save()


        ### DEFINING BOUNDING BOX

        def dimensions():
            bbox = BBox(bbox = BBOX, crs = CRS.WGS84)
            print("Bounding Box created")
            
            return bbox


        ### SENTINELHUB API WCS REQUEST FOR FAPAR LAYER
        ### FAPAR --> FRACTION OF ABSORBED PHOTOSYNTHETICALLY ACTIVE RADIATION

        def fapar_layer_call(bbox, input_date):
            wcs_true_color_request = WcsRequest(
                data_collection = DataCollection.SENTINEL2_L2A,
                data_folder = '/SENTINEL_PRODUCTS/FAPAR/',
                layer = 'FAPAR',
                bbox = bbox,
                time = input_date,
                resx = '10m',
                resy = '10m',
                image_format = MimeType.TIFF,
                custom_url_params = {
                    CustomUrlParam.SHOWLOGO: False
                },
                config = config
            )
            wcs_true_color_img = wcs_true_color_request.get_data()
            FAPAR = wcs_true_color_img[-1]

            print("FAPAR fetched")
            
            return FAPAR


        ### SENTINELHUB API WCS REQUEST FOR LSWI LAYER
        ### LSWI --> LAND SURFACE WATER INDEX

        def lswi_layer_call(bbox, input_date):
            wcs_true_color_request = WcsRequest(
                data_collection = DataCollection.SENTINEL2_L2A,
                data_folder = '/SENTINEL_PRODUCTS/LSWI/',
                layer = 'LSWI',
                bbox = bbox,
                time = input_date,
                resx = '10m',
                resy = '10m',
                image_format = MimeType.TIFF,
                custom_url_params = {
                    CustomUrlParam.SHOWLOGO: False
                },
                config = config
            )
            wcs_true_color_img = wcs_true_color_request.get_data()
            LSWI = wcs_true_color_img[-1]

            print("LSWI fetched")
            
            return LSWI


        ### SENTINEL DATA DICTINOARY

        def sentinel_data_dict(bbox, input_date):
            FAPAR = fapar_layer_call(bbox, input_date)
            LSWI = lswi_layer_call(bbox, input_date)

            sentinel_data = {
                'FAPAR' : FAPAR,
                'LSWI' : LSWI
            }

            return sentinel_data


        ### WATER STRESS SCALAR (W) CALCULATIONS

        def w_calc(sentinel_data):
            LSWI = sentinel_data['LSWI']
            LSWI_max = np.amax(LSWI)
            w = (1 - LSWI) / (1 + LSWI_max)

            print("Water Stres Scalar calculated")
            
            return w


        ### LIGHT USE EFFICIENCY (LUE) CALCULATIONS

        def lue_calc(sentinel_data):
            e0 = 3.22          # MAXIMUM VALUE OF LUE
            w = w_calc(sentinel_data)
            LUE = e0 * w

            print("LUE calculated")
            
            return LUE


        ### GLOBAL HORIZONTAL IRRADIANCE CALCULATIONS

        def ghi_calc(input_date, bbox):
            lat = bbox.middle[1]
            lon = bbox.middle[0]
            API_KEY = 'eb8cf696ffe68ba33f4b7c3b25e45d5c' #OpenWeatherMap @ARMS4AI
            #-------Call this API Once-------
            response = requests.get(f"https://api.openweathermap.org/energy/1.0/solar/data?lat={lat}&lon={lon}&date={input_date}&appid={API_KEY}")
            data = response.json()
        
            GHI = 0
            for i in data["irradiance"]["daily"]:
                GHI = i['clear_sky']['ghi']
            GHI = (GHI * 3.6) / 1000
            
            return GHI


        ### NET PRIMARY PRODUCTIVITY (NPP) CALCULATIONS

        def npp_calc(input_date, sentinel_data, bbox):
            FAPAR = sentinel_data['FAPAR']
            GHI = ghi_calc(input_date, bbox)
            LUE = lue_calc(sentinel_data)
            NPP = FAPAR * GHI * 0.5 * LUE
            NPP[NPP < 0] = 0
            print("NPP calculated")
            
            return NPP


        ### HARVEST INDEX DICTIONARY

        def harvest_index_dict(crop):
            harvest_index = {
                "Potato" : 0.2,
                "Cotton" : 0.008,
                "Wheat" : 0.01,
                "Corn" : 0.045,
                "Barley" : 0.01,
                "Sunflower" : 0.01,
                "Sugarcane" : 0.98,
                "Chilli"    : 0.017
            }
            
            return harvest_index[crop]


        ### CROP YIELD CALCULATIONS

        def cy_calc(input_date, crop, sentinel_data, bbox):
            NPP = npp_calc(input_date, sentinel_data, bbox)
            HI = harvest_index_dict(crop)         
            cy = NPP * HI * 10
            print(np.amin(cy), np.amax(cy))
            
            return cy


        ### CLIPPING RASTER

        def clipping_raster(geojson_data, cy_path):
            with rasterio.open(cy_path) as src:   
                out_image, out_transform = rasterio.mask.mask(src, geojson_data.geometry, crop = True)
                out_meta = src.meta.copy()
                out_meta.update({"driver": "GTiff",
                                "height": out_image.shape[1],
                                "width": out_image.shape[2],
                                "transform": out_transform})
                
            with rasterio.open(cy_path, "w", **out_meta) as dest:
                dest.write(out_image)

        ### ZONAL STATISTICS 

        def zonal_stats_calc(geojson_data, cy_path):
            cy = zonal_stats(geojson_data,
                            cy_path,
                            band = 1,
                            nodata = np.nan,
                            stats = ['mean'], 
                            geojson_out = True)
            cy_mean_dict = {}
            for i in range(len(cy)):
                cy_mean_dict[cy[i]['id']] = round(cy[i]['properties']['mean'], 2)
            
            print(cy_mean_dict)
                
            cy_mean = round(sum(cy_mean_dict.values()) / len(cy_mean_dict), 2)
            return cy, cy_mean


        ### EXCEL GENERATION

        def excel(cy):
            cy_excel = pd.DataFrame()
            for i in range(len(cy)):
                cy_excel = cy_excel.append(cy[i]['properties'], ignore_index=True)

            return cy_excel


        ### GEOJSON GENERATION

        def geojson(geojson_data, cy):
            geojson_data['Crop_Yield'] = " "
            for i in range(len(geojson_data)):
                geojson_data['Crop_Yield'][i] = cy['mean'][i]

            return geojson_data


        ### MAIN FUNCTION

        
        input_date, crop = input_data()
        bbox_tmp = dimensions()
        print(bbox_tmp)
        sentinel_data = sentinel_data_dict(bbox_tmp, input_date)
        transform = rasterio.transform.from_bounds(BBOX[0], BBOX[1], BBOX[2], BBOX[3], sentinel_data['FAPAR'].shape[1], sentinel_data['FAPAR'].shape[0])

        cy_path = 'CY.tiff'
        cy = cy_calc(input_date, crop, sentinel_data, bbox_tmp)
        with rasterio.open(cy_path, 'w', driver = 'GTiff', width = cy.shape[1], height = cy.shape[0], count = 1, dtype = cy.dtype, crs = 'EPSG:4326', transform = transform) as dst:
            dst.write(cy, 1)

        clipping_raster(geojson_data, cy_path)
        cy, cy_mean = zonal_stats_calc(geojson_data, cy_path)
        cy = excel(cy)
        geojson_data = geojson(geojson_data, cy)

        cy.to_excel('/OUTPUT/cy.xlsx')
        geojson_data.to_file('/OUTPUT/cy.geojson')
        with open('/OUTPUT/cy_mean.json', 'w') as outfile:
            json.dump(cy_mean, outfile)
    
        print('Crop Yield calculated')

response = cropYeild(Date, BBOX)

tool_outputs = []
tool_outputs.append({"tool_call_id":tool_calls[0].id, "output": response})

crop-yield-script
Input Data fetched 
date: 2024-06-09
crop: Sugarcane
Bounding Box created
80.118914,27.826019,80.180197,27.885513
FAPAR fetched
LSWI fetched
Water Stres Scalar calculated
LUE calculated
NPP calculated
0.0 183.50577


UnboundLocalError: cannot access local variable 'geojson_data' where it is not associated with a value

In [43]:
pwd

'd:\\GitHub\\agentic_rag_llamaIndex_tutorial\\basics'

In [41]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=tool_outputs
)

In [42]:
import time
while run.status not in ["completed", "failed","requires_action"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [43]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
for msg in messages:
    print(msg.content[0].text.value)

The number of homeless people in ABC in 2022 is 123.
How many people were homeless in ABC in 2022?


In [23]:
from openai import OpenAI

# Set your OpenAI API key
OpenAI.api_key = "px7kW2QzMmQS9O24hXmoT3BlbkFJfF2dRzmdi41AXfrl6zy6"

# Define the prompt template
prompt_template = """
Calculate the {crop_type} yield at the {latitude}° N, {longitude}° E for date {date}.

Input:
{input}

Output:
"""

# Define a list of examples to help the model understand the task
examples = [
    "rice yield at the 10.5014° N, 76.2758° E for date 1-1-2023",
    "wheat yield at the 29.9792° N, 78.1390° E for date 15-2-2023",
    "corn yield at the 32.7157° N, 117.1611° W for date 12-3-2023",
]

# Create the prompt by combining the examples and the prompt template
prompt = prompt_template.format(
    crop_type="sugarcane",
    latitude="28.7041",
    longitude="77.1025",
    date="2-12-2023",
    input="\n".join(examples),
)

# Send the prompt to the OpenAI API and get the response
response = OpenAI().chat.completions.create(
    engine="text-davinci-003",
    prompt=prompt,
    temperature=0,
    max_tokens=100,
    n=1,
    stop=None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

# Extract the response from the API
output = response.choices[0].text.strip()

# Check if the required parameters are present in the user input
required_params = ["crop_type", "latitude", "longitude", "date"]
input_params = {param: None for param in required_params}

for param in input_params:
    if param in output:
        input_params[param] = output.split(param)[1].split(" ")[0].strip()

# If any parameter is missing, ask the user to provide it
for param in required_params:
    if input_params[param] is None:
        print(f"Please provide the {param} parameter.")
        input_params[param] = input(f"Enter {param}: ")

# Create the final prompt with the user-provided parameters
final_prompt = prompt_template.format(
    crop_type=input_params["crop_type"],
    latitude=input_params["latitude"],
    longitude=input_params["longitude"],
    date=input_params["date"],
    input="\n".join(examples + [output]),
)

# Send the final prompt to the OpenAI API and get the response
response = OpenAI().chat.completions.create(
    engine="text-davinci-003",
    prompt=final_prompt,
    temperature=0,
    max_tokens=100,
    n=1,
    stop=None,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

# Extract the response from the API
output = response.choices[0].text.strip()

print(output)

TypeError: Missing required arguments; Expected either ('messages' and 'model') or ('messages', 'model' and 'stream') arguments to be given